In [1]:
import pandas as pd
import geopandas as gpd
from summaflow import (
    GeoLayer,
    SUMMAWorkflow,
    Stats,
)

import os
import glob

In [2]:
# paths
root_path = '../../bb-model-lumped//'

# layers' paths
landcover_path = os.path.join(root_path, 'attributes', 'landcover', 'MCD12Q1.061')
soilclass_path = os.path.join(root_path, 'attributes', 'soil')
merithdyr_path = os.path.join(root_path, 'attributes', 'elevation')

# geolayer's path
riv_path = os.path.join(root_path, 'shapefiles', 'bb_rivers_lumped.shp')
cat_path = os.path.join(root_path, 'shapefiles', 'bb_subbasins_lumped.shp')
hru_path = os.path.join(root_path, 'shapefiles', 'bb_subbasins_lumped.shp')

# forcings path
root_path_forcings = os.path.join(root_path, 'forcing', 'remapped')

In [3]:
# Geospatial layers
riv_obj = gpd.read_file(riv_path)
cat_obj = gpd.read_file(cat_path)
hru_obj = gpd.read_file(hru_path)

In [4]:
# layers needed by the setup workflow
# elevation
elv = GeoLayer.from_maf(
    maf_stats=os.path.join(merithdyr_path, 'bb_model_lumped_stats_elv.csv'),
    maf_layer=os.path.join(merithdyr_path, 'bb_model_lumped_elv.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'meters',
)
# landcover
landcover = GeoLayer.from_maf(
    maf_stats=os.path.join(landcover_path, 'bb_model_lumped_stats_MCD12Q1.061_2022.csv'),
    maf_layer=os.path.join(landcover_path, 'bb_model_lumped_2022.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)
# USDA soil classes
soil = GeoLayer.from_maf(
    maf_stats=os.path.join(soilclass_path, 'bb_model_lumped_stats_soil_classes.csv'),
    maf_layer=os.path.join(soilclass_path, 'bb_model_lumped_soil_classes.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)

# custom layers for `tan_slope`, `contourLength` and `downHRUindex`
# until relevant workflows are implemented inside `gistool`--sorry
# For now, look at various constructors for "GeoLayer"
slope = GeoLayer( # workflow needs `mean` stat
    stats=Stats(pd.DataFrame([0.1] * len(cat_obj), index=cat_obj['COMID'], columns=['mean'])),
    unit='dimensionless',
)
contour = GeoLayer( # workflow needs `length` stat
    stats=Stats(
        pd.DataFrame(
            cat_obj.set_crs(epsg=4326).to_crs('ESRI:54009').length, index=cat_obj['COMID'], columns=['length'])),
    unit='meter',
)
hru_index = GeoLayer( # workflow needs `index` "stat"
    stats=Stats(pd.DataFrame([0] * len(cat_obj), index=cat_obj['COMID'], columns=['index'])),
    unit='dimensionless',
)

In [5]:
exp = SUMMAWorkflow(
    forcing_data = glob.glob(os.path.join(root_path_forcings, '**', '*.nc'), recursive=True),
    forcing_name_mapping = {
        'CaSR_v3.1_A_PR0_SFC': 'pptrate',
        'CaSR_v3.1_P_TT_09975': 'airtemp',
        'CaSR_v3.1_P_P0_SFC': 'airpres',
        'CaSR_v3.1_P_FI_SFC': 'LWRadAtm',
        'CaSR_v3.1_P_FB_SFC': 'SWRadAtm',
        'CaSR_v3.1_P_HU_09975': 'spechum',
        'CaSR_v3.1_P_UVC_09975': 'windspd',
    },
    forcing_unit_mapping = {
        'pptrate': 'meter / hour',
        'airtemp': 'degC',
        'airpres': 'millibar',
        'LWRadAtm': 'watt / meter ** 2',
        'SWRadAtm': 'watt / meter ** 2',
        'spechum': 'dimensionless',
        'windspd': 'knot',
    },
    forcing_to_unit_mapping = {
        'pptrate': 'millimeter / second',
        'airtemp': 'kelvin',
        'airpres': 'pascal',
        'LWRadAtm': 'watt / meter ** 2',
        'SWRadAtm': 'watt / meter ** 2',
        'spechum': 'dimensionless',
        'windspd': 'meter / second',
    },
    forcing_attrs = {
        'measurement_height': 20,
        'measurement_height_unit': 'meter',
        'forcing_time_zone': 'utc', # original timezone of the forcing datatset
        'target_time_zone': 'utc', # if UTC, SUMMA converts to local time zone internally
        'local': {},
        'global': {},
    },
    topology_data = {
        'riv': riv_obj,
        'hru': hru_obj,
        'cat': cat_obj,
    },
    topology_unit_mapping = {}, # not sure if mizuRoute should be included here
    topology_to_unit_mapping = {}, # not sure if mizuRoute should be included here
    topology_attrs = {
        'gru_fid': 'COMID',
        'hru_fid': 'COMID',
        'local': {},
        'global': {},
    },
    geospatial_data = {
        'elevation': elv,
        'soilTypeIndex': soil,
        'vegTypeIndex': landcover,
        'tan_slope': slope,
        'contourLength': contour,
        'downHRUindex': hru_index,
    },
    cold_state = {
        'layers': {
            'nSoil': 8,
            'nSnow': 0,
        },
        'states': { # dimension manipulation is automated inside the workflow
            'scalarCanopyIce': 0,
            'scalarCanopyLiq': 0,
            'scalarSnowDepth': 0,
            'scalarSWE': 0,
            'scalarSfcMeltPond': 0,
            'scalarAquiferStorage': 0.4,
            'scalarSnowAlbedo': 0,
            'scalarCanairTemp': 283.16,
            'scalarCanopyTemp': 283.16,
            'mLayerTemp': 283.16,
            'mLayerVolFracIce': 0,
            'mLayerVolFracLiq': 0.4,
            'mLayerMatricHead': -1.0,
            'mLayerDepth': [0.025, 0.075, 0.15, 0.25, 0.5, 0.5, 1, 1.5],
        },
    },
    decisions = { # Can change all decisions, otherwise default values
        'soilCatTbl': 'ROSETTA',
    },
    auxillary = {
        # 'dt_init': 450 # if not provided, defaults to forcing data timestep
    },
    settings = {
        'model_path': os.path.join(root_path, 'settings', 'SUMMA'),
        'start_date': '1980-01-01 00:00',
        'end_date': '1980-01-10 23:00',
        'verbose': True,
    },
    fillna = {
        'geospatial_data': {
            'elevation': 1, # a rough assumption--can be modified to anything
            'soilTypeIndex': 6, # based on Darri's assumption--can be modified to anything
            'vegTypeIndex': 1, # a rough assumption--can be modified to anything
        },
    },
)

2025-06-02 11:55:25,463 - summaflow.core - INFO - SUMMA workflow initialized


/home/kasra.keshavarz1/virtual-envs/summaflow/lib/python3.11/site-packages/summaflow/core.py:267: UserWarning: `dt_init` not provided in auxillary dictionary; defaulting to forcing time-step.
  warnings.warn("`dt_init` not provided in auxillary dictionary;"


## Workflow tests

In [6]:
exp.run(path=os.path.join(root_path), save=True)

2025-06-02 11:55:25,467 - summaflow.core - INFO - Running SUMMA workflow
2025-06-02 11:55:25,892 - summaflow.core - INFO - Initializing attributes for SUMMA workflow...
2025-06-02 11:55:25,893 - summaflow.core - INFO - Creating attributes xarray.Dataset
2025-06-02 11:55:25,894 - summaflow.core - INFO - Adding `mHeight` attribute
2025-06-02 11:55:25,895 - summaflow.core - INFO - Adding `slopeTypeIndex` attribute
2025-06-02 11:55:25,896 - summaflow.core - INFO - Adding `hruId` and `gruId` attributes
2025-06-02 11:55:25,898 - summaflow.core - INFO - Adding `hru2gruId` attributes
2025-06-02 11:55:25,901 - summaflow.core - INFO - Calculating and adding `latitude` and `logitude` attributes
2025-06-02 11:55:25,909 - summaflow.core - INFO - Calculating and adding `area` attributes


/home/kasra.keshavarz1/virtual-envs/summaflow/lib/python3.11/site-packages/summaflow/utils.py:138: UserWarning: Using EPSG:3857 for centroid calculations on geographic data. For better accuracy, specify a local projection using calculation_crs parameter.
  warnings.warn(


2025-06-02 11:55:26,170 - summaflow.core - INFO - Adding geospatial layers' attributes
2025-06-02 11:55:26,170 - summaflow.core - INFO - Adding `tan_slope` attributes
2025-06-02 11:55:26,172 - summaflow.core - INFO - Adding `contourLength` attributes
2025-06-02 11:55:26,173 - summaflow.core - INFO - Adding `downHRUindex` attributes
2025-06-02 11:55:26,175 - summaflow.core - INFO - Adding `elevation` attributes
2025-06-02 11:55:26,176 - summaflow.core - INFO - Adding `vegTypeIndex` attributes
2025-06-02 11:55:26,178 - summaflow.core - INFO - Adding `soilTypeIndex` attributes
2025-06-02 11:55:26,182 - summaflow.core - INFO - Adding local and global attributes of the Dataset
2025-06-02 11:55:26,183 - summaflow.core - INFO - SUMMA attributes initialized successfully.
2025-06-02 11:55:26,184 - summaflow.core - INFO - Saving dataset to /home/kasra.keshavarz1/github-repos/bow-river-at-banff/bb-model-lumped/settings/SUMMA/attributes.nc
2025-06-02 11:55:26,484 - summaflow.core - INFO - Initiali